In [1]:
##This Baseline Model using Afinn to calculate the average sentiment per tweet for input and using linear regression as the model
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
from collections import defaultdict
from collections import OrderedDict
import datetime as DT
import ast
import numpy as np
from string import punctuation
import urllib.request, json 

from afinn import Afinn
af = Afinn()

In [2]:
##Translate date of the tweet to quarter reference
def Quarterlookup(date):
    if date>=DT.datetime(2018,10,1):
        return '18Q4'
    elif date>=DT.datetime(2018,7,1):
        return '18Q3'
    elif date>=DT.datetime(2018,4,1):
        return '18Q2'
    elif date>=DT.datetime(2018,1,1):
        return '18Q1'
    elif date>=DT.datetime(2017,10,1):
        return '17Q4'
    elif date>=DT.datetime(2017,7,1):
        return '17Q3'
    elif date>=DT.datetime(2017,4,1):
        return '17Q2'
    elif date>=DT.datetime(2017,1,1):
        return '17Q1'
    elif date>=DT.datetime(2016,10,1):
        return '16Q4'
    elif date>=DT.datetime(2016,7,1):
        return '16Q3'
    elif date>=DT.datetime(2016,4,1):
        return '16Q2'
    elif date>=DT.datetime(2016,1,1):
        return '16Q1'
    elif date>=DT.datetime(2015,10,1):
        return '15Q4'
    elif date>=DT.datetime(2015,7,1):
        return '15Q3'
    elif date>=DT.datetime(2015,4,1):
        return '15Q2'
    else:
        return '15Q1'

In [23]:
##Reading the Input X file and process it
d=defaultdict(list)
with open('InputX.txt') as f:
    for line in f:
        count=0
        temp =-1
        k,v=line.split(':',1)
        
        while True:
       
            oldtemp=temp
            temp=v.find(']]',temp+1)
            if count==0:
                d[k].append(v[oldtemp+2:temp+2]) 
            else:
                d[k].append(v[oldtemp+4:temp+2])
           
             
            count+=1
            if temp == -1: break
            #if count == 10000: break  # initially limit for time sake
     
  

In [24]:
##Converting the string of data into dictionary
InputX=defaultdict(dict)
count=0

for key,value in d.items():
    Inputx=defaultdict(list)

    for value2 in value:
        
        temp=value2.find('), ')
        date=value2[14:temp+3]
        try:
            year=int(date[1:5])
            month=int(date[date.find(',')+2:date.find(',',date.find(',')+1)])
            day=int(date[date.find(')')-2:date.find(')')])
            sentiment_scoreaf=0
      
            ##Calculating the average sentiment score
            for i in ast.literal_eval(value2[temp+3:-1]):     
                sentiment_scoreaf+=af.score(i)
  
            Inputx[Quarterlookup(DT.datetime(year,month,day))].append(sentiment_scoreaf)
            count+=1
        except:
            pass
    InputX[key]=OrderedDict(sorted(Inputx.items()))


In [25]:
##List of stockname and relevant stock code
l = [("CVSHealth", "CVS"),
("RiteAid", "RAD"),
("Conn's", "CONN"),
("Casey's", "CASY"),
("Kroger", "KR"),
("WeisMarkets", "WMK"),
("HavertyFurniture", "HVT"),
("Kirkland's", "KIRK"),
("Pier1Imports","PIR"),
("BuildersFirstSource","BLDR"),
("HuttigBuildingProducts", "HBP"),
("LumberLiquidators", "LL"),
("TransWorldEntertainment", "TWMC"),
("MarineMax", "HZO"),
("Medifast", "MED"),
("SallyBeauty", "SBH"),
("Sotheby's", "BID"),
("StarGasPartners", "SGU"),
("SuburbanPropanePartners", "SPH"),
("TitanMachinery", "TITN"),
("TractorSupply","TSCO"),
("TravelCentersofAmerica", "TA"),
("Big5SportingGoods", "BGFV"),
("Dick'sSportingGoods", "DKS"),
("HibbettSports", "HIBB"),
("Build-A-BearWorkshop", "BBW")]
     
all_stock = {}
[all_stock.update({k:v}) for k,v in l]
print(all_stock)

{'CVSHealth': 'CVS', 'RiteAid': 'RAD', "Conn's": 'CONN', "Casey's": 'CASY', 'Kroger': 'KR', 'WeisMarkets': 'WMK', 'HavertyFurniture': 'HVT', "Kirkland's": 'KIRK', 'Pier1Imports': 'PIR', 'BuildersFirstSource': 'BLDR', 'HuttigBuildingProducts': 'HBP', 'LumberLiquidators': 'LL', 'TransWorldEntertainment': 'TWMC', 'MarineMax': 'HZO', 'Medifast': 'MED', 'SallyBeauty': 'SBH', "Sotheby's": 'BID', 'StarGasPartners': 'SGU', 'SuburbanPropanePartners': 'SPH', 'TitanMachinery': 'TITN', 'TractorSupply': 'TSCO', 'TravelCentersofAmerica': 'TA', 'Big5SportingGoods': 'BGFV', "Dick'sSportingGoods": 'DKS', 'HibbettSports': 'HIBB', 'Build-A-BearWorkshop': 'BBW'}


In [26]:
##Read the quarterly return detail from edgaronline and convert into library
top="http://datafied.api.edgar-online.com/v2/corefinancials/qtr?primarysymbols="
bottom="&appkey=asc97xrhkyu4959aptu76zxj"
Outputy =defaultdict(dict)
for stockname,stock in all_stock.items():
    check=top+stock+bottom
    with urllib.request.urlopen(check) as url:
        data = json.loads(url.read().decode())
    count1=0
    count2=0
    end_date=[]
    revenue_change=[]
    last_rev=0

    for key, value in data.items():
        
        for key2, value2 in value.items():          
        
            if count1!=0:  
                for i in reversed(value2):
                  
                    for j in i["values"]:
                     
                        if count2!=0:   
                            if j["field"] =="periodenddate":
                                end_date.append(Quarterlookup(DT.datetime.strptime(j['value'], '%m/%d/%Y')))
                       
                            ##Calculate change of revenue into the y_label
                            if j["field"] =="totalrevenue":
                                revenue_cal =round(float(j['value'])/float(last_rev) - 1,3)
                                last_revQ4=j['value']
                                if revenue_cal>0:
                                    revenue_change.append(1)
                                else:
                                    revenue_change.append(0)
                             
                        else:
                            ##Storing each of the initial revenue for calculation
                            if j["field"] =="totalrevenue":      
                                last_rev=j['value']
                              
                    count2+=1
            count1+=1
            
    ##Output will be a dictionary with stockname
    Outputy[stockname]["period"]=end_date
    Outputy[stockname]["revenuechange"]=revenue_change
        

In [27]:
##Calculating the training data
stock =['CVSHealth', 'RiteAid', "Conn's", "Casey's", 'Kroger', 'WeisMarkets', 'HavertyFurniture', "Kirkland's", 'Pier1Imports', 'BuildersFirstSource', 'HuttigBuildingProducts', 'LumberLiquidators', 'TransWorldEntertainment', 'MarineMax', 'Medifast', 'SallyBeauty',"Sotheby's", 'StarGasPartners', 'SuburbanPropanePartners', 'TitanMachinery', 'TractorSupply', 'TravelCentersofAmerica', 'Big5SportingGoods', "Dick'sSportingGoods", 'HibbettSports', 'Build-A-BearWorkshop']
quarter=['15Q1','15Q2','15Q3','15Q4','16Q1','16Q2','16Q3','16Q4','17Q1','17Q2','17Q3','17Q4','18Q1']

X_value=[]
Y_value=[]
for i in stock:
    for j in range(len(quarter)):
        X_value.append(np.mean(InputX[i][quarter[j]]))
        Y_value.append(Outputy[i]['revenuechange'][j])

In [28]:
##Calculating the testing data
teststock =['CVSHealth', 'RiteAid', "Conn's", "Casey's", 'Kroger', 'WeisMarkets', 'HavertyFurniture', "Kirkland's", 'Pier1Imports', 'BuildersFirstSource', 'HuttigBuildingProducts', 'LumberLiquidators', 'TransWorldEntertainment', 'MarineMax', 'Medifast', 'SallyBeauty',"Sotheby's", 'StarGasPartners', 'SuburbanPropanePartners', 'TitanMachinery', 'TractorSupply', 'TravelCentersofAmerica', 'Big5SportingGoods', "Dick'sSportingGoods", 'HibbettSports', 'Build-A-BearWorkshop']
testquarter=['18Q3']

X_valuetest=[]
Y_valuetest=[]
for i in teststock:
    for j in range(len(testquarter)):
        X_valuetest.append(np.mean(InputX[i][quarter[j]]))
        Y_valuetest.append(Outputy[i]['revenuechange'][j])

In [29]:
##Calculating correlation
print(np.corrcoef(X_value,Y_value)[0, 1])

0.1465015987998694


In [30]:
##Creating the Linear Regression Model 
import statsmodels.api as sm

model = sm.OLS(Y_value, X_value)
results2 = model.fit()
print(results2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.313
Model:                            OLS   Adj. R-squared:                  0.311
Method:                 Least Squares   F-statistic:                     153.5
Date:                Sat, 08 Dec 2018   Prob (F-statistic):           2.60e-29
Time:                        23:51:20   Log-Likelihood:                -297.00
No. Observations:                 338   AIC:                             596.0
Df Residuals:                     337   BIC:                             599.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.2961      0.024     12.391      0.0

In [31]:
##Calculating Accuracy
Right=0
Total=0
for i in range(len(X_valuetest)):
    if X_valuetest[i]>0:
        if Y_valuetest[i]==1:
            Right+=1
            Total+=1
        else:
            Total+=1
    else:
        if Y_valuetest[i]==0:
            Right+=1
            Total+=1
        else:
            Total+=1



print('Accuracy score: {0:0.2f}'.format(
      Right/Total))

Accuracy score: 0.42


In [32]:
##Calculating precision
from sklearn.metrics import average_precision_score

y_pred=[]
for i in X_valuetest:
    if i>0:
        y_pred.append(1)
    else:
        y_pred.append(0)
    

average_precision = average_precision_score(y_pred,Y_valuetest)

print('Average precision score: {0:0.2f}'.format(
      average_precision))

Average precision score: 0.83
